In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

seed = 2023
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.experimental.numpy.random.seed(seed)
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
os.environ["TF_DETERMINISTIC_OPS"] = "1"
# Set a fixed value for the hash seed
os.environ["PYTHONHASHSEED"] = str(seed)
import dolfin as df
import time
from utils_training import *
from utils import *

import seaborn as sns

sns.set_theme()
sns.set_context("paper")
sns.set(rc={"xtick.bottom": True, "ytick.left": True})
colors = sns.color_palette("mako").as_hex()
my_cmap = sns.color_palette("viridis", as_cmap=True)
%load_ext autoreload
%autoreload 2


In [ ]:
original_size = 10000
n_mode = 4


def compute_residues(X, Y):
    F = X[:, :, :, 0] * max_norm_F
    Phi = X[:, :, :, 1]
    G = X[:, :, :, 2]
    W = Y[:, :, :, 0]
    domain = X[:, :, :, -1]
    laplacian = laplacian_fn(Phi * W + G)
    error = tf.reduce_mean(
        ((laplacian + F) ** 2 * domain),
        axis=[1, 2],
    )
    magnitude = tf.reduce_mean(((F) ** 2 * domain), axis=[1, 2])
    residues_interior = error / magnitude

    return residues_interior


dtype = tf.float32
F = tf.constant(np.load(f"./data_{original_size}/F.npy"), dtype=dtype)
max_norm_F = tf.constant(
    np.max(np.sqrt(np.mean(F**2, axis=(1, 2)))), dtype=dtype
)
print("max_norm_F :", max_norm_F.numpy())

F = tf.constant(
    np.load(f"./data_{original_size}/F.npy") / max_norm_F,
    dtype=dtype,
)
G = tf.constant(
    np.load(f"./data_{original_size}/G.npy"),
    dtype=dtype,
)
Phi = tf.constant(np.load(f"./data_{original_size}/Phi_64.npy"), dtype=dtype)
Y = tf.constant(np.load(f"./data_{original_size}/W.npy"), dtype=dtype)[
    :, :, :, None
]

data_size = F.shape[0]


nb_vert = F.shape[1]
domain, boundary = create_domain_and_boundary(nb_vert, Phi)
domain, boundary = tf.constant(domain, dtype=dtype), tf.constant(
    boundary, dtype=dtype
)

X = tf.stack([F, Phi, G, domain], axis=3)
input_shape = (None, X.shape[1], X.shape[2], X.shape[3])

residues_interior = compute_residues(X, Y)

residues_interior_val = residues_interior


In [ ]:
print(f"{np.argmax(residues_interior)=}")
print(f"{np.min(residues_interior)=}")
print(f"{np.max(residues_interior)=}")
print(f"{np.mean(residues_interior)=}")

residues_interior_argsort = np.argsort(residues_interior)
residues_interior_sort = np.sort(residues_interior)

test = np.array([residues_interior <= 1e-3])
print(f"{test.sum()=}")

residues_interior_argsort = np.argsort(residues_interior)
residues_interior_sort = np.sort(residues_interior)

print(
    f"{np.max(residues_interior.numpy()[residues_interior_argsort[:test.sum()]])=}"
)
print(
    f"{np.min(residues_interior.numpy()[residues_interior_argsort[:test.sum()]])=}"
)


In [ ]:
def plot_data(i):
    phi = tf.constant(
        np.load(f"./data_{original_size}/Phi_64.npy"), dtype=tf.float32
    )[i]
    F = tf.constant(
        np.load(f"./data_{original_size}/F.npy"), dtype=tf.float32
    )[i]
    W = tf.constant(
        np.load(f"./data_{original_size}/W.npy"), dtype=tf.float32
    )[i]
    G = tf.constant(
        np.load(f"./data_{original_size}/G.npy"), dtype=tf.float32
    )[i]
    domain, boundary = domain_and_border(np.shape(phi)[0], phi)

    domains_nan = domain
    domains_nan[domains_nan == 0] = np.nan
    plt.figure(figsize=(12, 3))
    plt.subplot(1, 4, 1)
    plt.imshow(F * domains_nan, cmap=my_cmap, origin="lower")
    plt.title(r"$F$")
    plt.colorbar(shrink=0.6)
    plt.subplot(1, 4, 2)
    plt.imshow(
        -laplacian_fn(phi[None, :, :] * W[None, :, :] + G[None, :, :])[0, :, :]
        * domains_nan,
        cmap=my_cmap,
        origin="lower",
    )
    plt.colorbar(shrink=0.6)
    plt.title(r"$-\Delta \phi w$")
    plt.subplot(1, 4, 3)
    plt.imshow(W * domains_nan * phi, cmap=my_cmap, origin="lower")
    plt.colorbar(shrink=0.6)
    plt.title(r"$U$")
    plt.subplot(1, 4, 4)
    plt.imshow(phi, cmap=my_cmap, origin="lower")
    plt.colorbar(shrink=0.6)
    plt.title(r"$\phi$")
    plt.tight_layout()
    plt.show()


In [ ]:
for i in list(residues_interior_argsort[-5:]):
    print(f"i = {i}")
    print(f"{residues_interior.numpy()[i]=}")
    plot_data(i)
    print("########")


In [ ]:
original_params = np.load(f"./data_{original_size}/agentParams.npy")
original_F = np.load(f"./data_{original_size}/F.npy")
original_W = np.load(f"./data_{original_size}/W.npy")
original_Phi = np.load(f"./data_{original_size}/Phi.npy")
original_Phi_64 = np.load(f"./data_{original_size}/Phi_64.npy")
original_G = np.load(f"./data_{original_size}/G.npy")
original_level_set = np.load(
    f"./data_domains_{original_size}_{n_mode}/level_sets_{original_size}.npy"
)
original_domains = np.load(
    f"./data_domains_{original_size}_{n_mode}/domains_{original_size}.npy"
)
original_params_ls = np.load(
    f"./data_domains_{original_size}_{n_mode}/params_{original_size}.npy"
)


print(f"{np.shape(original_params)=}")
print(f"{np.shape(original_F)=}")
print(f"{np.shape(original_W)=}")
print(f"{np.shape(original_Phi)=}")
print(f"{np.shape(original_Phi_64)=}")
print(f"{np.shape(original_G)=}")
print(f"{np.shape(original_level_set)=}")
print(f"{np.shape(original_domains)=}")
print(f"{np.shape(original_params_ls)=}")

print(f"{np.shape(residues_interior_argsort)=}")
print(f"{np.shape(residues_interior_argsort[: test.sum()])=}")
print(f"{np.shape(original_F[residues_interior_argsort[: test.sum()]])=}")
print(f"{np.shape(original_W[residues_interior_argsort[: test.sum()]])=}")
print(f"{np.shape(original_params[residues_interior_argsort[: test.sum()]])=}")
print(f"{np.shape(original_Phi[residues_interior_argsort[: test.sum()]])=}")
print(f"{np.shape(original_Phi_64[residues_interior_argsort[: test.sum()]])=}")
print(f"{np.shape(original_G[residues_interior_argsort[: test.sum()]])=}")
print(
    f"{np.shape(original_level_set[residues_interior_argsort[: test.sum()]])=}"
)
print(
    f"{np.shape(original_domains[residues_interior_argsort[: test.sum()]])=}"
)
print(
    f"{np.shape(original_params_ls[residues_interior_argsort[: test.sum()]])=}"
)

new_F, new_W, new_params, new_Phi, new_Phi_64, new_G = (
    original_F[residues_interior_argsort[: test.sum()]],
    original_W[residues_interior_argsort[: test.sum()]],
    original_params[residues_interior_argsort[: test.sum()]],
    original_Phi[residues_interior_argsort[: test.sum()]],
    original_Phi_64[residues_interior_argsort[: test.sum()]],
    original_G[residues_interior_argsort[: test.sum()]],
)

new_level_set, new_domains, new_params_ls = (
    original_level_set[residues_interior_argsort[: test.sum()]],
    original_domains[residues_interior_argsort[: test.sum()]],
    original_params_ls[residues_interior_argsort[: test.sum()]],
)


randomize = np.arange(len(new_F))

np.random.shuffle(randomize)

new_F = new_F[randomize]
new_W = new_W[randomize]
new_params = new_params[randomize]
new_Phi = new_Phi[randomize]
new_Phi_64 = new_Phi_64[randomize]
new_G = new_G[randomize]
new_level_set = new_level_set[randomize]
new_domains = new_domains[randomize]
new_params_ls = new_params_ls[randomize]

print(f"{np.shape(new_params)=}")
print(f"{np.shape(new_F)=}")
print(f"{np.shape(new_W)=}")
print(f"{np.shape(new_Phi)=}")
print(f"{np.shape(new_Phi_64)=}")
print(f"{np.shape(new_G)=}")
print(f"{np.shape(new_level_set)=}")
print(f"{np.shape(new_domains)=}")
print(f"{np.shape(new_params_ls)=}")


new_size = new_F.shape[0]
print(f"{new_size=}")
if not (os.path.exists(f"./data_{new_size}/")):
    os.makedirs(f"./data_{new_size}/")


np.save(f"./data_{new_size}/agentParams.npy", new_params)
np.save(f"./data_{new_size}/F.npy", new_F)
np.save(f"./data_{new_size}/W.npy", new_W)
np.save(f"./data_{new_size}/Phi.npy", new_Phi)
np.save(f"./data_{new_size}/Phi_64.npy", new_Phi_64)
np.save(f"./data_{new_size}/G.npy", new_G)
np.save(f"./data_{new_size}/params_ls.npy", new_params_ls)
np.save(f"./data_{new_size}/level_sets.npy", new_level_set)
np.save(f"./data_{new_size}/domains.npy", new_domains)